In [1]:
%pip install torch torchvision transformers
%pip install wandb

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [2]:
import os
import json
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from PIL import Image
import torch
import torch.nn as nn
from modules import UNet

class ImageCaptionDataset(Dataset):
    def __init__(self, img_dir, seg_dir, caption_file, transform=None):
        self.img_dir = img_dir
        self.seg_dir = seg_dir
        self.transform = transform
        with open(caption_file, 'r') as f:
            self.captions = json.load(f)
        self.filenames = list(self.captions.keys())

    def __len__(self):
        return len(self.filenames)

    def __getitem__(self, idx):
        img_name = self.filenames[idx]
        seg_name = 'segmented_' + img_name  #Adjusting name for segmented images

        img_path = os.path.join(self.img_dir, img_name)
        seg_path = os.path.join(self.seg_dir, seg_name)

        image = Image.open(img_path).convert('RGB')
        segmentation = Image.open(seg_path).convert('L')

        if self.transform:
            image = self.transform(image)
            segmentation = self.transform(segmentation)

        caption = self.captions[img_name]
        return image, segmentation, caption


# Transformations
transform = transforms.Compose([
    transforms.CenterCrop((512, 512)),
    transforms.Resize((512, 512)),
    transforms.ToTensor(),
])

/Users/princepatel/mit/school/6.8300/cv-finalproject/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
import torch
import torch.nn.functional as F
import cv2
import numpy as np

def apply_sobel_operator(segmentation):
    sobel_x = torch.tensor([[-1, 0, 1], [-2, 0, 2], [-1, 0, 1]], dtype=torch.float32).view(1, 1, 3, 3)
    sobel_y = torch.tensor([[-1, -2, -1], [0, 0, 0], [1, 2, 1]], dtype=torch.float32).view(1, 1, 3, 3)
    
    # Ensure the segmentation tensor is float and has a batch dimension
    if len(segmentation.shape) == 2:
        segmentation = segmentation.unsqueeze(0).unsqueeze(0)
    elif len(segmentation.shape) == 3:
        segmentation = segmentation.unsqueeze(1)

    edges_x = F.conv2d(segmentation, sobel_x, padding=1)
    edges_y = F.conv2d(segmentation, sobel_y, padding=1)

    edges = torch.sqrt(edges_x**2 + edges_y**2)

    threshold = edges.mean() * 1.5
    binary_mask = (edges > threshold).float()

    return binary_mask

def apply_canny_edge_detector(segmentation, low_threshold=100, high_threshold=200):
    """
    Applies the Canny edge detector to a PyTorch tensor representing an image and returns a binary mask of the edges.

    Args:
        image_tensor (torch.Tensor): A tensor representing the image. Expected shape (B, H, W) where B is the size of batch.
        low_threshold (int): Lower threshold for the hysteresis process in Canny.
        high_threshold (int): Higher threshold for the hysteresis process in Canny.

    Returns:
        torch.Tensor: A binary mask where edges are 1 and non-edges are 0, as a tensor.
    """
    segmentation = segmentation * 255
    batch_np = segmentation.numpy().astype(np.uint8)
    
    # Initialize an empty list to collect edge maps
    edge_maps = []
    
    # Process each image in the batch
    for image_np in batch_np:
        blurred_image = cv2.GaussianBlur(image_np, (3, 3), 0)
        edges = cv2.Canny(blurred_image, low_threshold, high_threshold)
        edge_maps.append(edges)
    
    # Convert list of arrays back to a tensor
    edge_tensor = torch.from_numpy(np.stack(edge_maps, axis=0).astype(np.uint8))
    return edge_tensor

class Params:
    def __init__(self):
        self.batch_size = 64
        self.learning_rate = 0.001
        self.epochs = 10

In [4]:
from transformers import CLIPProcessor, CLIPModel

class DiffusionModel(nn.Module):
    def __init__(self, beta_start, beta_end, steps):
        super(DiffusionModel, self).__init__()
        # Networks definition
        self.delta_network = nn.Sequential(
            nn.Linear(512, 256),
            nn.ReLU(),
            nn.Linear(256, 1)
        )
        self.recovery_network = UNet(c_in=3, c_out=3, time_dim=256, device="cpu") 
        self.beta_t = torch.linspace(beta_start, beta_end, steps=steps)
        self.alpha_t = torch.cumprod(1 - self.beta_t, dim=0)

        self.clip_processor = CLIPProcessor.from_pretrained('openai/clip-vit-base-patch32')
        self.text_encoder = CLIPModel.from_pretrained('openai/clip-vit-base-patch32')


    def compute_delta(self, text_features, t):
        # Compute the delta value based on text features and timestep
        delta_scale = self.delta_network(text_features).squeeze()
        return delta_scale

    def forward_pass(self, x_t_minus_1, m, captions, t):
        # Process captions through the text encoder (CLIP)
        inputs = self.clip_processor(text=captions, return_tensors="pt", padding=True, truncation=True).to(self.recovery_network.device)
        text_features = self.text_encoder.get_text_features(**inputs)  # Using the appropriate method to get text features from CLIP

        alpha_t = self.alpha_t[t]
        one_minus_alpha_t = 1 - alpha_t

        # Compute delta
        delta = self.compute_delta(text_features, t)
        delta = delta.unsqueeze(-1).unsqueeze(-1).unsqueeze(-1)

        # Generate noise
        weighted_noise = torch.randn_like(x_t_minus_1) * delta

        # Compute the next timestep value
        x_t = torch.sqrt(alpha_t) * x_t_minus_1 + torch.sqrt(one_minus_alpha_t) * weighted_noise
        return x_t

    def reverse_pass(self, x_t, m, t):
        # Compute recovery parameters
        t = torch.tensor(t, dtype=torch.float32, device='cpu')
        recovery_params = self.recovery_network(x_t, t)
        
        x_t_minus_1 = (recovery_params * (1-m)) + (x_t * m)
        return x_t_minus_1

In [5]:
def diffusion_loss(x_t, x_t1_pred, m):
    return ((x_t1_pred - x_t) ** 2 * (1 - m)).mean()

In [6]:
import wandb
from sklearn.model_selection import KFold
import torch
from torch.utils.data import DataLoader

# Setup
T = 1000  # Total number of diffusion steps


n_splits = 5
kf = KFold(n_splits=n_splits, shuffle=True, random_state=42)
num_epochs = 10

# Initialize W&B
# wandb.init(project='image_captioning_with_diffusion_models', entity='michaelpeng72', config={
#     "epochs": num_epochs,
#     "batch_size": 16,
#     "learning_rate": 1e-4,
#     "n_splits": n_splits,
#     "shuffle": True,
#     "random_state": 42
# })

# config = wandb.config
config = Params()

full_dataset = ImageCaptionDataset('datasets/images', 'datasets/segmented', 'datasets/map.json', transform=transform)

for fold, (train_idx, valid_idx) in enumerate(kf.split(full_dataset)):
    print(f"Starting Fold {fold+1}/{n_splits}")
    # wandb.init(project='image_captioning_with_diffusion_models', entity='michaelpeng72', group="Experiment-X", job_type=f"Fold-{fold+1}", reinit=True)

    # Data loaders
    train_subset = torch.utils.data.Subset(full_dataset, train_idx)
    valid_subset = torch.utils.data.Subset(full_dataset, valid_idx)
    train_loader = DataLoader(train_subset, batch_size=config.batch_size, shuffle=True)
    valid_loader = DataLoader(valid_subset, batch_size=config.batch_size, shuffle=False)

    # Initialize model and optimizer
    model = DiffusionModel(beta_start=0.0001, beta_end=0.02, steps=1000)
    optimizer = torch.optim.Adam(model.parameters(), lr=config.learning_rate)

    # Training
    model.train()
    for epoch in range(config.epochs):
        train_loss = 0
        for images, segmentations, captions in train_loader:
            # m = apply_sobel_operator(segmentations)
            m = apply_canny_edge_detector(segmentations)
            t = torch.randint(0, T, (1,)).item()

            # Forward diffusion process
            noised_images = model.forward_pass(images, m, captions, t)

            # Reverse diffusion process
            recovered_images = model.reverse_pass(noised_images, m, t)

            # Calculate loss
            loss = diffusion_loss(images, recovered_images, m)
            train_loss += loss.item()

            # Backpropagation
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

        train_loss /= len(train_loader)
        # wandb.log({"Train Loss": train_loss})

        # Validation
        model.eval()
        valid_loss = 0
        with torch.no_grad():
            for images, segmentations, captions in valid_loader:
                # m = apply_sobel_operator(segmentations)
                m = apply_canny_edge_detector(segmentations)
                t = torch.randint(0, T, (1,)).item()
                
                # Same forward and reverse pass for validation
                noised_images = model.forward_pass(images, m, captions, t)
                recovered_images = model.reverse_pass(noised_images, m, captions, t)

                # Calculate validation loss
                loss = diffusion_loss(images, recovered_images, m)
                valid_loss += loss.item()

        valid_loss /= len(valid_loader)
        # wandb.log({"Validation Loss": valid_loss})
        print(f"Epoch {epoch+1}, Train Loss: {train_loss:.4f}, Validation Loss: {valid_loss:.4f}")
        
    # wandb.finish()


Starting Fold 1/5


/Users/princepatel/mit/school/6.8300/cv-finalproject/.venv/lib/python3.12/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


: 

In [ ]:
#save the model to ../models
torch.save(model.state_dict(), 'models/diffusion_model.pth')

In [ ]:
#load model from ../models
model = DiffusionModel()
model.load_state_dict(torch.load('models/diffusion_model.pth'))

In [ ]:
import matplotlib.pyplot as plt

example_segmented =  Image.open('datasets/segmented/segmented_COCO_val2014_000000000459.jpg').convert('RGB')
example_segmented = transform(example_segmented)

plt.imshow(example_segmented[0], cmap='gray')
plt.show()
print(example_segmented)
# m = apply_sobel_operator(example_segmented)
m = apply_canny_edge_detector(example_segmented, 10, 110)

#plot m
plt.imshow(m[0].numpy(), cmap='gray')
plt.show()